In [7]:
import meshplot as mp
import numpy as np

import xml.etree.ElementTree as ET

def load_color_profile(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    vertices = [float(x) for x in root.find("Tags").find("gamutBoundaryDescType").find("Vertices").find("PCSValues").text.split()]
    vertices = np.reshape(vertices, (-1, 3))
    print(vertices)
    triangles = np.array([[int(x) for x in t.text.split()] for t in root.find("Tags").find("gamutBoundaryDescType").find("Triangles")])
    print(triangles)
    return vertices, triangles

def load_gam_file(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
        vertices = []
        triangles = []
        begin_count = 0
        is_read = False
        for line in lines:
            tokens = line.split()
            if not tokens: 
                continue
            if tokens[0] == "BEGIN_DATA":
                begin_count += 1
                is_read = True
                continue
            elif tokens[0] == "END_DATA":
                is_read = False
                continue
            if begin_count == 1 and is_read:
                vertices.append([float(x) for x in tokens[1:]])
            elif begin_count == 2 and is_read:
                triangles.append([int(x) for x in tokens])
    return np.array(vertices), np.array(triangles)


def create_colors(vertices):
    colors = np.zeros((vertices.shape[0], 3))
    # convert CIE L*a*b* to RGB
    for i, vertex in enumerate(vertices):
        L, a, b = vertex
        # convert to XYZ
        Y = (L + 16) / 116
        X = a / 500 + Y
        Z = Y - b / 200
        X = 0.95047 * (X ** 3 if X ** 3 > 0.008856 else (X - 16 / 116) / 7.787)
        Y = 1.00000 * (Y ** 3 if Y ** 3 > 0.008856 else (Y - 16 / 116) / 7.787)
        Z = 1.08883 * (Z ** 3 if Z ** 3 > 0.008856 else (Z - 16 / 116) / 7.787)
        # convert to RGB
        R = X * 3.2406 + Y * -1.5372 + Z * -0.4986
        G = X * -0.9689 + Y * 1.8758 + Z * 0.0415
        B = X * 0.0557 + Y * -0.2040 + Z * 1.0570
        R = (12.92*R) if R <= 0.0031308 else 1.055 * (R ** (1 / 2.4)) - 0.055
        G = (12.92*G) if G <= 0.0031308 else 1.055 * (G ** (1 / 2.4)) - 0.055
        B = (12.92*B) if B <= 0.0031308 else 1.055 * (B ** (1 / 2.4)) - 0.055
        colors[i] = [R, G, B]
    return colors
        

def main():
    # vertices, triangles = load_color_profile("profiles/sRGB_D65_MAT.xml")
    vertices, triangles = load_gam_file("profiles/AdobeRGB1998.gam")
    colors = create_colors(vertices)
    mp.plot(vertices, triangles,c=colors, return_plot=True)

if __name__ == "__main__":
    main()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(50.0, -18…

In [8]:
import 

SyntaxError: invalid syntax (2494350585.py, line 1)